In [13]:
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 18 20:18:15 2020

@author: jeffr
"""

# packages needed
import time
#import selenium as sl
#from selenium import webdriver
import pandas as pd
from pytube import YouTube


In [14]:
# import dataframe of powerhour videos
df = pd.read_csv('./powerhour.csv')

print(df.head(5))

   number        clipid  year          event  \
0       1  HorseRacing1  2015  Triple crown    
1       2         Golf1  2019        Masters   
2       3         Golf7  2017           Open   
3       4         Golf9  2016      Ryder Cup   
4       5        Golf10  2016        Masters   

                                                desc  \
0                                   American Pharaoh   
1  Tiger wins 2019 masters (water balls by compet...   
2                            Spieth Open go get that   
3                   Reed-Rory Ryder cup singles 2016   
4                                    Spieth collapse   

                                                link  duration  
0  https://www.youtube.com/watch?v=WyDcXtVb8_U&t=101        60  
1    https://www.youtube.com/watch?v=pE-aOBGN8UM&t=0        70  
2    https://www.youtube.com/watch?v=-0q3doVGUuE&t=0        80  
3    https://www.youtube.com/watch?v=3fKB0UJxhe4&t=0        68  
4  https://www.youtube.com/watch?v=bMf0JpfpZu8&t=

In [58]:
# create iterables for video loops
itrlink = df.link
itrduration = df.duration

df = df[0:2]

print(df.iloc[0].link)

VIDEO_STAGE_PATH = './video_staging/'

test_link = itrlink[0]

https://www.youtube.com/watch?v=WyDcXtVb8_U&t=101


In [61]:
#  download youtube videos
for clip in range(0,len(df)):
    
    yt = YouTube(df.iloc[clip].link)

    mp4files = yt.streams.filter('mp4')

    yt.set_filename = df.iloc[clip].clipid

    d_video = yt.streams.first()#(mp4files[-1].extension,mp4files[-1].resolution)

    try: 
        #downloading the video 
        d_video.download(VIDEO_STAGE_PATH) 
    except: 
        print("Some Error!") 
    print('Task Completed!')

Task Completed!
Task Completed!
